In [ ]:
abstract type Term end
abstract type Formula end

@auto_hash_equals struct PredicateSymbol
    symbol::String
end

@auto_hash_equals struct FunctionSymbol
    symbol::String
end

@auto_hash_equals struct Variable <: Term
    variable::String
end

@auto_hash_equals struct Function <: Term
    name::FunctionSymbol
    arguments::Vector{Term}
end

@auto_hash_equals struct AtomicFormula <: Formula
    name::PredicateSymbol
    arguments::Vector{Term}
end

@auto_hash_equals struct Negation <: Formula
    formula::Formula
end

@auto_hash_equals struct EFormula <: Formula
    variable::Variable
    formula::Formula
end

@auto_hash_equals struct AFormula <: Formula
    variable::Variable
    formula::Formula
end

@auto_hash_equals struct Conjunction <: Formula
    formula1::Formula
    formula2::Formula
end

@auto_hash_equals struct Disjunction <: Formula 
    formula1::Formula
    formula2::Formula
end

@auto_hash_equals struct Literal
    negative::Bool
    formula::AtomicFormula
end

In [13]:
using Catlab

@theory Category{Ob,Hom} begin





end


LoadError: Base.Meta.ParseError("Ill-formed theory binding Category{Ob, Hom}")

In [16]:

abstract type Term end

#variables
struct Sym <: Term
    name
end

struct Lit <: Term
    name
end

struct Func <: Term
    head
    args
end




struct ForAll <: Term
    vars
    body
end

struct Exists <: Term
    vars
    body
end

struct And <: Term
    args 
end

struct Or <: Term
    args
end

struct Not <: Term
    body
end

struct Eq <: Term
    left
    right
end

struct NotEq <: Term
    left
    right
end

struct Implies <: Term
    hyp
    conc
end

struct Equiv <: Term
    left
    right
end

In [15]:
dump(:(compose(f,compose(g,h))))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol compose
    2: Symbol f
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol compose
        2: Symbol g
        3: Symbol h


In [19]:
#Maybe we should just idrectly use stuff?
[1,2,3][2:end]

2-element Array{Int64,1}:
 2
 3

In [43]:

# macro ized hoas
dump(:(forall( (x, y) -> p(x, forall(x -> z(x))))))

Expr
  head: Symbol call
  args: Array{Any}((2,))
    1: Symbol forall
    2: Expr
      head: Symbol ->
      args: Array{Any}((2,))
        1: Expr
          head: Symbol tuple
          args: Array{Any}((2,))
            1: Symbol x
            2: Symbol y
        2: Expr
          head: Symbol block
          args: Array{Any}((2,))
            1: LineNumberNode
              line: Int64 1
              file: Symbol In[43]
            2: Expr
              head: Symbol call
              args: Array{Any}((3,))
                1: Symbol p
                2: Symbol x
                3: Expr


In [22]:
function tptp( x::Expr ) 
    if x.head == :call
        return "$(x.args[1])($(join(map(tptp,x.args[2:end]),",")))"
    end
end
tptp(x :: Symbol) = String(x)


tptp (generic function with 2 methods)

In [25]:
tptp(:(compose(f,compose(g,h))))
repr(:(compose(f,compose(g,h)))) # I'm very close to being able to just do this. But we probably want the 

":(compose(f, compose(g, h)))"

In [5]:
And( x::T, y::S ) where {T <: Term, S <: Term} = And([x,y])
Or( x::T, y::S ) where {T <: Term, S <: Term} = Or([x,y])
Func(n::String) = Func(n,[])

Or

In [20]:
tptp( x::Not  ) = "( ~ $(tptp(x.body)) )"
tptp( x::Eq  ) = "($(tptp(x.left)) = $(tptp(x.right)))"
tptp( x::NotEq  ) = "($(tptp(x.left)) != $(tptp(x.right)))"
tptp( x::Implies  ) = "( $(tptp(x.hyp)) => $(tptp(x.conc)) )"
tptp( x::ForAll ) = "(! [$(join(map(tptp,x.vars),","))] : $(tptp(x.body)))"
tptp( x::Exists ) = "(? [$(join(map(tptp,x.vars),","))] : $(tptp(x.body)))"
tptp( x::And ) = "($(join(map(tptp,x.args)," & ")))"
tptp( x::Or ) = "($(join(map(tptp,x.args)," | ")))"
tptp( x::Sym ) = uppercasefirst(String(x.name))
tptp( x::Lit ) = String(x.name)
tptp( f::Func ) = "$(f.head)($(join(map(tptp,f.args),",")))"


tptp (generic function with 11 methods)

ErrorException: expression is not a function call, or is too complex for @code_llvm to analyze; break it down to simpler parts if possible

In [23]:
#methods(|)
Base.:|( x::T, y::S ) where {T <: Term, S <: Term} = Or(x,y)
Base.:&( x::T, y::S ) where {T <: Term, S <: Term} = And(x,y)

Base.:!( x::T) where {T <: Term} = Not(x)
=>( x::T, y::S ) where {T <: Term, S <: Term} = Implies(x,y)
#Base.:<=>( x::T, y::S ) where {T <: Term, S <: Term} = Equiv(x,y)

==( x::T, y::S ) where {T <: Term, S <: Term} = Eq(x,y)
import Base.!=
!=( x::T, y::S ) where {T <: Term, S <: Term} = NotEq(x,y)
(f::Func)(x...) = Func(f.head, vcat(f.args , [y for y in x]))

In [8]:
x = Sym(:x)
y = Sym(:y)
t = ForAll([x,x],  Exists([y], x == y => x  ))
tptp(t)

"(! [X,X] : (? [Y] : ( (X = Y) => X )))"

In [37]:
ty = Func("ty", [])

Func("ty", Any[])

In [56]:
tptp(ty(x,x,x))

"ty(X,X,X)"

In [ ]:

struct Axiom
    name
    formula
end
struct Theory
    axioms
end



Func

In [24]:
lives = Func("lives")
killed = Func("killed")
richer = Func("richer")
hates = Func("hates")
x = Sym(:X)
y = Sym(:Y)
butler = Lit("butler")
agatha = Lit("agatha")
charles = Lit("charles")


Lit("charles")

In [15]:
t = Exists([x], lives(x) & killed(x, agatha) )
tptp(t)

"(? [X] : (lives(X) & killed(X,agatha)))"

In [31]:
agatha_theory = Dict("e1" => lives(agatha),
"e2" => lives(butler),
"e3" => lives(charles),
"e4" => ForAll([x], lives(x) => (   (x == agatha) | (x == charles) | (x == butler ))   ),
"e5" => ForAll([x,y], killed(x,y) => hates(x,y) ) 
)



MethodError: MethodError: no method matching =>(::String, ::Func)
You may have intended to import Base.:(=>)
Closest candidates are:
  =>(!Matched::T, ::S) where {T<:Term, S<:Term} at In[23]:6

In [47]:
agatha_theory = [Exists([x], lives(x) & killed(x, agatha) ),
lives(agatha),
lives(butler),
lives(charles),
ForAll([x], lives(x) => (   (x == agatha) | (x == charles) | (x == butler ))   ),
ForAll([x,y], killed(x,y) => hates(x,y) ) ,


ForAll([x,y], killed(x,y) => !richer(x,y) ), 
ForAll([x], hates(agatha,x) => !hates(charles,x) ), 
ForAll([x], x != butler => hates(agatha,x) ) ,
ForAll([x], !richer(x,agatha) => hates(butler,x) ), 
ForAll([x], hates(agatha,x) => hates(butler,x) ),
ForAll([x], Exists([y], !hates(x,y))),
agatha != butler]

map(tptp, agatha_theory)

conjecture = killed(agatha,agatha)





Func("killed", Any[Lit("agatha"), Lit("agatha")])

In [60]:
query = join([ "fof(e$i , axiom, $(tptp(f)) )."   for (i,f) in  enumerate(agatha_theory)], "\n") *  "\nfof(gfoo, conjecture, $(tptp(conjecture)))." 

"fof(e1 , axiom, (? [X] : (lives(X) & killed(X,agatha))) ).\nfof(e2 , axiom, lives(agatha) ).\nfof(e3 , axiom, lives(butler) ).\nfof(e4 , axiom, lives(charles) ).\nfof(e5 , axiom, (! [X] : ( lives(X) => (((X = agatha) | (X = charles)) | (X = butler)) )) ).\nfof(e6 , axiom, (! [X,Y] : ( killed(X,Y) => hates(X,Y) )) ).\nfof(e7 , axiom, (! [X,Y] : ( killed(X,Y) => ( ~ richer(X,Y) ) )) ).\nfof(e8 , axiom, (! [X] : ( hates(agatha,X) => ( ~ hates(charles,X) ) )) ).\nfof(e9 , axiom, (! [X] : ( (X != butler) => hates(agatha,X) )) ).\nfof(e10 , axiom, (! [X] : ( ( ~ richer(X,agatha) ) => hates(butler,X) )) ).\nfof(e11 , axiom, (! [X] : ( hates(agatha,X) => hates(butler,X) )) ).\nfof(e12 , axiom, (! [X] : (? [Y] : ( ~ hates(X,Y) ))) ).\nfof(e13 , axiom, (agatha != butler) ).\nfof(gfoo, conjecture, killed(agatha,agatha))."

In [61]:

open(`eprover`, "w", stdout) do io
        print(io, query)      
       end

{
   no_preproc:                    false
   eqdef_maxclauses:              20000
   eqdef_incrlimit:               20
   heuristic_name:                Default
   heuristic_def:                 
   prefer_initial_clauses:        false
   selection_strategy:            NoSelection
   pos_lit_sel_min:               0
   pos_lit_sel_max:               9223372036854775807
   neg_lit_sel_min:               0
   neg_lit_sel_max:               9223372036854775807
   all_lit_sel_min:               0
   all_lit_sel_max:               9223372036854775807
   weight_sel_min:                0
   select_on_proc_only:           false
   inherit_paramod_lit:           false
   inherit_goal_pm_lit:           false
   inherit_conj_pm_lit:           false
   enable_eq_factoring:           true
   enable_neg_unit_paramod:       true
   enable_given_forward_simpl:    true
   pm_type:                       ParamodPlain
   ac_handling:                   1
   ac_res_aggressive:             true
   forward_co

In [62]:
pipeline( `echo $query`  , `eprover`)

pipeline(`echo 'fof(e1 , axiom, (? [X] : (lives(X) & killed(X,agatha))) ).
fof(e2 , axiom, lives(agatha) ).
fof(e3 , axiom, lives(butler) ).
fof(e4 , axiom, lives(charles) ).
fof(e5 , axiom, (! [X] : ( lives(X) => (((X = agatha) | (X = charles)) | (X = butler)) )) ).
fof(e6 , axiom, (! [X,Y] : ( killed(X,Y) => hates(X,Y) )) ).
fof(e7 , axiom, (! [X,Y] : ( killed(X,Y) => ( ~ richer(X,Y) ) )) ).
fof(e8 , axiom, (! [X] : ( hates(agatha,X) => ( ~ hates(charles,X) ) )) ).
fof(e9 , axiom, (! [X] : ( (X != butler) => hates(agatha,X) )) ).
fof(e10 , axiom, (! [X] : ( ( ~ richer(X,agatha) ) => hates(butler,X) )) ).
fof(e11 , axiom, (! [X] : ( hates(agatha,X) => hates(butler,X) )) ).
fof(e12 , axiom, (! [X] : (? [Y] : ( ~ hates(X,Y) ))) ).
fof(e13 , axiom, (agatha != butler) ).
fof(gfoo, conjecture, killed(agatha,agatha)).'`, stdout=`eprover`)

In [36]:
res = read(`eprover`, String))



E 2.5pre002 "Avongrove"

Usage: eprover [options] [files]

Read a set of first-order clauses and formulae and try to refute it.

Options:

   -h
  --help
    Print a short description of program usage and options.

   -V
  --version
    Print the version number of the prover. Please include this with all bug
    reports (if any).

   -v
  --verbose[=<arg>]
    Verbose comments on the progress of the program. This differs from the
    output level (below) in that technical information is printed to stderr,
    while the output level determines which logical manipulations of the
    clauses are printed to stdout. The short form or the long form without
    the optional argument is equivalent to --verbose=1.

   -o <arg>
  --output-file=<arg>
    Redirect output into the named file.

   -s
  --silent
    Equivalent to --output-level=0.

   -l <arg>
  --output-level=<arg>
    Select an output level, greater values imply more verbose output. Level 0
    produces nearly no output, level 1 w

In [ ]:
#variables
struct Sym
    name::Symbol
end

struct Term
    f::Symbol
    arguments::Array{Any} # Array{Union{Term,Sym}}
end


struct ForAll
    vars::Vec{Sym}
    body
end

struct Exists
    vars::Vec{Sym}
    body
end

struct And
    arguments::Vec{Any}
end

struct Or
    arguments::Vec{Any}
end

struct Not
    body
end

struct Eq
    left
    right
end

struct Implies
    hyp
    conc
end

struct Equiv
    left
    right
end


tptp( x::Not  ) = "( ~ $(tptp(x.body)) )"
tptp( x::Eq  ) = "( $(tptp(x.left)) = $(tptp(x.right)) )"
tptp( x ::ForAll ) = "( ! [  $(join(map(tptp,x.vars)," , "))  ] :  $(tptp(x.body)) )"
tptp( x ::Exists ) = "( ? [  $(join(map(tptp,x.vars)," , "))  ] :  $(tptp(x.body)) )"
tptp( x ::And ) = "( $(join(map(tptp,x.vars)," & ")) )"
tptp( x ::Or ) = "( $(join(map(tptp,x.vars)," | ")) )"


#=
\neg
¬
∧ \wedge \v \vee

∀
∃
=#




In [19]:
" $(1 + 2) $(3 + 4)"

" 3 7"

In [20]:
"( $(join(map(string,[1 2 3]),"&")) )"

"( 1&2&3 )"

In [18]:
string("x " ," y")

"x  y"

In [16]:
dump(:(a ∧ b ∧ c))
dump(:( ? [x]  ))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol ∧
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol ∧
        2: Symbol a
        3: Symbol b
    3: Symbol c


LoadError: syntax: invalid identifier name "?"

http://www.tptp.org/cgi-bin/SeeTPTP?Category=Problems&Domain=PUZ&File=PUZ131_1.p



tff(student_type,type,(
    student: $tType )).

tff(professor_type,type,(
    professor: $tType )).

tff(course_type,type,(
    course: $tType )).

tff(michael_type,type,(
    michael: student )).

tff(victor_type,type,(
    victor: professor )).

tff(csc410_type,type,(
    csc410: course )).

tff(enrolled_type,type,(
    enrolled: ( student * course ) > $o )).

tff(teaches_type,type,(
    teaches: ( professor * course ) > $o )).

tff(taught_by_type,type,(
    taughtby: ( student * professor ) > $o )).

tff(coordinator_of_type,type,(
    coordinatorof: course > professor )).

tff(student_enrolled_axiom,axiom,(
    ! [X: student] :
    ? [Y: course] : enrolled(X,Y) )).

tff(professor_teaches,axiom,(
    ! [X: professor] :
    ? [Y: course] : teaches(X,Y) )).

tff(course_enrolled,axiom,(
    ! [X: course] :
    ? [Y: student] : enrolled(Y,X) )).

tff(course_teaches,axiom,(
    ! [X: course] :
    ? [Y: professor] : teaches(Y,X) )).

tff(coordinator_teaches,axiom,(
    ! [X: course] : teaches(coordinatorof(X),X) )).

tff(student_enrolled_taught,axiom,(
    ! [X: student,Y: course] :
      ( enrolled(X,Y)
     => ! [Z: professor] :
          ( teaches(Z,Y)
         => taughtby(X,Z) ) ) )).

tff(michael_enrolled_csc410_axiom,axiom,(
    enrolled(michael,csc410) )).

tff(victor_coordinator_csc410_axiom,axiom,(
    coordinatorof(csc410) = victor )).

tff(teaching_conjecture,conjecture,(
    taughtby(michael,victor) )).
